## Level 1

In [6]:
import typing as tp


def solution(queries: tp.List[str]):
    db_ = {}

    def set_or_inc(key, field, value):
        if key not in db_:
            db_[key] = {field: int(value)}

        elif field not in db_[key]:
            db_[key].update({field: int(value)})

        else:
            db_[key][field] += int(value)

        return str(db_[key][field])

    def get(key, field):
        if key not in db_:
            return ""

        if field not in db_[key]:
            return ""

        return str(db_[key][field])

    def delete(key, field):
        if key not in db_:
            return "false"

        if field not in db_[key]:
            return "false"

        db_[key].pop(field)
        if not db_[key]:
            del db_[key]
        return "true"

    result = []
    for q in queries:
        operation = q[0]

        if operation == "SET_OR_INC":
            result.append(set_or_inc(q[1], q[2], q[3]))
        elif operation == "GET":
            result.append(get(q[1], q[2]))
        elif operation == "DELETE":
            result.append(delete(q[1], q[2]))

    return result


queries = [
    ["SET_OR_INC", "A", "B", "5"],
    ["SET_OR_INC", "A", "B", "6"],
    ["GET", "A", "B"],
    ["GET", "A", "C"],
    ["DELETE", "A", "B"],
    ["DELETE", "A", "C"],
]
result = solution(queries)
print(result)

delete
['5', '11', '11', '', 'true', 'false']


## Level 2

In [8]:
import typing as tp
from collections import defaultdict


def solution(queries: tp.List[str]):
    db_ = {}
    modifs_ = defaultdict(int)

    def set_or_inc(key, field, value):
        if key not in db_:
            db_[key] = {field: int(value)}

        elif field not in db_[key]:
            db_[key].update({field: int(value)})

        else:
            db_[key][field] += int(value)

        modifs_[key] += 1

        return str(db_[key][field])

    def get(key, field):
        if key not in db_:
            return ""

        if field not in db_[key]:
            return ""

        return str(db_[key][field])

    def delete(key, field):
        if key not in db_:
            return "false"

        if field not in db_[key]:
            return "false"

        db_[key].pop(field)
        modifs_[key] += 1

        if not db_[key]:
            del db_[key]
            del modifs_[key]

        return "true"

    def top_n_keys(topk):
        candidates = sorted(modifs_.items(), key=lambda item: (-item[1], item[0]))[: int(topk)]
        return ", ".join([f"{t[0]}({t[1]})" for t in candidates])

    result = []
    for q in queries:
        operation = q[0]

        if operation == "SET_OR_INC":
            result.append(set_or_inc(q[1], q[2], q[3]))
        elif operation == "GET":
            result.append(get(q[1], q[2]))
        elif operation == "DELETE":
            result.append(delete(q[1], q[2]))
        elif operation == "TOP_N_KEYS":
            result.append(top_n_keys(q[1]))

    return result


queries = [
    ["SET_OR_INC", "C", "field1", "10"],
    ["TOP_N_KEYS", "5"],
    ["SET_OR_INC", "A", "field1", "5"],
    ["DELETE", "A", "field1"],
    ["SET_OR_INC", "B", "field1", "8"],
    ["SET_OR_INC", "B", "field1", "0"],
    ["TOP_N_KEYS", "3"],
    ["DELETE", "B", "field2"],
    ["SET_OR_INC", "B", "field2", "6"],
    ["SET_OR_INC", "A", "field1", "4"],
    ["SET_OR_INC", "A", "field1", "2"],
    ["TOP_N_KEYS", "2"],
    ["TOP_N_KEYS", "3"],
]
result = solution(queries)
print(result)

['10', 'C(1)', '5', 'true', '8', '8', 'B(2), C(1)', 'false', '6', '4', '6', 'B(3), A(2)', 'B(3), A(2), C(1)']


## Level 3

In [14]:
import typing as tp
from collections import defaultdict, deque


def solution(queries: tp.List[str]):
    db_ = {}
    modifs_ = defaultdict(int)
    locks_ = defaultdict(deque)

    def set_or_inc(key, field, value):
        if key not in db_:
            db_[key] = {field: int(value)}
            modifs_[key] += 1
            return str(db_[key][field])

        if key not in locks_:
            if field not in db_[key]:
                db_[key].update({field: int(value)})
            else:
                db_[key][field] += int(value)
            modifs_[key] += 1
            return str(db_[key][field])

        return str(db_[key][field]) if field in db_[key] else "0"

    def get(key, field):
        if key not in db_:
            return ""

        if field not in db_[key]:
            return ""

        return str(db_[key][field])

    def delete(key, field):
        if key not in db_:
            return "false"

        if field not in db_[key]:
            return "false"

        if key in locks_:
            return "false"

        db_[key].pop(field)
        modifs_[key] += 1

        if not db_[key]:
            del db_[key]
            del modifs_[key]

        return "true"

    def top_n_keys(topk):
        candidates = sorted(modifs_.items(), key=lambda item: (-item[1], item[0]))[: int(topk)]
        return ", ".join([f"{t[0]}({t[1]})" for t in candidates])

    def set_or_inc_by_caller(key, field, value, caller_id):
        if key not in db_:
            db_[key] = {field: int(value)}
            modifs_[key] += 1
            return str(db_[key][field])

        if key not in locks_:
            if field not in db_[key]:
                db_[key].update({field: int(value)})
            else:
                db_[key][field] += int(value)

            modifs_[key] += 1
            return str(db_[key][field])

        if locks_[key][0] == caller_id:
            if field not in db_[key]:
                db_[key].update({field: int(value)})
            else:
                db_[key][field] += int(value)

            modifs_[key] += 1
            return str(db_[key][field])

        return str(db_[key][field]) if field in db_[key] else "0"

    def delete_by_caller(key, field, caller_id):
        if key not in db_:
            return "false"

        if field not in db_[key]:
            return "false"

        if key in locks_ and locks_[key][0] != caller_id:
            # locked by others
            return "false"

        db_[key].pop(field)
        modifs_[key] += 1

        if not db_[key]:
            del db_[key]
            del modifs_[key]
            del locks_[key]

        return "true"

    def lock(caller_id, key):
        if key not in db_:
            return "invalid_request"
        elif key not in locks_:
            locks_[key].append(caller_id)
            return "acquired"
        else:
            if locks_[key][0] == caller_id:
                return ""
            elif caller_id in locks_[key]:
                return ""
            else:
                locks_[key].append(caller_id)
                return "wait"

    def unlock(key):
        if key not in db_:
            return "invalid_request"
        elif key not in locks_:
            return ""
        else:
            locks_[key].popleft()
            return "released"

    result = []
    for q in queries:
        operation = q[0]

        if operation == "SET_OR_INC":
            result.append(set_or_inc(q[1], q[2], q[3]))
        elif operation == "GET":
            result.append(get(q[1], q[2]))
        elif operation == "DELETE":
            result.append(delete(q[1], q[2]))
        elif operation == "TOP_N_KEYS":
            result.append(top_n_keys(q[1]))
        elif operation == "SET_OR_INC_BY_CALLER":
            result.append(set_or_inc_by_caller(q[1], q[2], q[3], q[4]))
        elif operation == "DELETE_BY_CALLER":
            result.append(delete_by_caller(q[1], q[2], q[3]))
        elif operation == "LOCK":
            result.append(lock(q[1], q[2]))
        elif operation == "UNLOCK":
            result.append(unlock(q[1]))

    return result


queries = [
    #
    # ["SET_OR_INC", "A", "B", "4"],
    # ["LOCK", "user1", "A"],
    # ["LOCK", "user2", "A"],
    # ["LOCK", "user3", "B"],
    # ["UNLOCK", "B"],
    # ["SET_OR_INC", "A", "C", "5"],
    # ["DELETE", "A", "B"],
    # ["SET_OR_INC_BY_CALLER", "A", "B", "3", "user2"],
    # ["GET", "A", "B"],
    # ["DELETE_BY_CALLER", "A", "B", "user3"],
    # ["SET_OR_INC_BY_CALLER", "A", "B", "5", "user1"],
    # ["UNLOCK", "A"],
    # ["SET_OR_INC_BY_CALLER", "A", "B", "2", "user1"],
    # ["SET_OR_INC_BY_CALLER", "A", "B", "1", "user2"],
    #
    ["SET_OR_INC", "A", "B", "4"],
    ["LOCK", "user1", "A"],
    ["LOCK", "user2", "A"],
    ["LOCK", "user3", "A"],
    ["UNLOCK", "A"],
    ["SET_OR_INC", "A", "C", "5"],
    ["DELETE", "A", "B"],
    ["SET_OR_INC_BY_CALLER", "A", "B", "3", "user3"],
    ["SET_OR_INC_BY_CALLER", "A", "B", "2", "user2"],
    ["GET", "A", "B"],
    ["GET", "A", "C"],
    ["DELETE_BY_CALLER", "A", "B", "user3"],
    ["SET_OR_INC_BY_CALLER", "A", "B", "5", "user1"],
    ["UNLOCK", "A"],
    ["SET_OR_INC_BY_CALLER", "A", "C", "2", "user3"],
    ["SET_OR_INC_BY_CALLER", "A", "B", "1", "user2"],
    ["GET", "A", "B"],
    ["GET", "A", "C"],
]
result = solution(queries)
print(result)

['4', 'acquired', 'wait', 'wait', 'released', '0', 'false', '4', '6', '6', '', 'false', '6', 'released', '2', '6', '6', '2']


## Level 4

In [ ]:
import typing as tp
from collections import defaultdict, deque


def solution(queries: tp.List[str]):
    db_ = {}
    modifs_ = defaultdict(int)
    locks_ = defaultdict(deque)

    def set_or_inc(key, field, value):
        if key not in db_:
            db_[key] = {field: int(value)}
            modifs_[key] += 1
            return str(db_[key][field])

        if key not in locks_:
            if field not in db_[key]:
                db_[key].update({field: int(value)})
            else:
                db_[key][field] += int(value)
            modifs_[key] += 1
            return str(db_[key][field])

        return str(db_[key][field]) if field in db_[key] else "0"

    def get(key, field):
        if key not in db_:
            return ""

        if field not in db_[key]:
            return ""

        return str(db_[key][field])

    def delete(key, field):
        if key not in db_:
            return "false"

        if field not in db_[key]:
            return "false"

        if key in locks_:
            return "false"

        db_[key].pop(field)
        modifs_[key] += 1

        if not db_[key]:
            del db_[key]
            del modifs_[key]

        return "true"

    def top_n_keys(topk):
        candidates = sorted(modifs_.items(), key=lambda item: (-item[1], item[0]))[: int(topk)]
        return ", ".join([f"{t[0]}({t[1]})" for t in candidates])

    def set_or_inc_by_caller(key, field, value, caller_id):
        if key not in db_:
            db_[key] = {field: int(value)}
            modifs_[key] += 1
            return str(db_[key][field])

        if key not in locks_:
            if field not in db_[key]:
                db_[key].update({field: int(value)})
            else:
                db_[key][field] += int(value)

            modifs_[key] += 1
            return str(db_[key][field])

        if locks_[key][0] == caller_id:
            if field not in db_[key]:
                db_[key].update({field: int(value)})
            else:
                db_[key][field] += int(value)

            modifs_[key] += 1
            return str(db_[key][field])

        return str(db_[key][field]) if field in db_[key] else "0"

    def delete_by_caller(key, field, caller_id):
        if key not in db_:
            return "false"

        if field not in db_[key]:
            return "false"

        if key in locks_ and locks_[key][0] != caller_id:
            # locked by others
            return "false"

        db_[key].pop(field)
        modifs_[key] += 1

        if not db_[key]:
            del db_[key]
            del modifs_[key]
            del locks_[key]

        return "true"

    def lock(caller_id, key):
        if key not in db_:
            return "invalid_request"
        elif key not in locks_:
            locks_[key].append(caller_id)
            return "acquired"
        else:
            if locks_[key][0] == caller_id:
                return ""
            elif caller_id in locks_[key]:
                return ""
            else:
                locks_[key].append(caller_id)
                return "wait"

    def unlock(key):
        if key not in db_:
            return "invalid_request"
        elif key not in locks_:
            return ""
        else:
            locks_[key].popleft()
            return "released"

    result = []
    for q in queries:
        operation = q[0]

        if operation == "SET_OR_INC":
            result.append(set_or_inc(q[1], q[2], q[3]))
        elif operation == "GET":
            result.append(get(q[1], q[2]))
        elif operation == "DELETE":
            result.append(delete(q[1], q[2]))
        elif operation == "TOP_N_KEYS":
            result.append(top_n_keys(q[1]))
        elif operation == "SET_OR_INC_BY_CALLER":
            result.append(set_or_inc_by_caller(q[1], q[2], q[3], q[4]))
        elif operation == "DELETE_BY_CALLER":
            result.append(delete_by_caller(q[1], q[2], q[3]))
        elif operation == "LOCK":
            result.append(lock(q[1], q[2]))
        elif operation == "UNLOCK":
            result.append(unlock(q[1]))

    return result


queries = [
    [""],
]
result = solution(queries)
print(result)